# Job description notebook
## Data cleaning and feaature_engineering

### Import libraries and read the csv file.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

jobs = pd.read_csv('glassdoor_jobs.csv', index_col=[0])

jobs.info()

jobs.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 956 entries, 0 to 955
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Job Title          956 non-null    object 
 1   Salary Estimate    956 non-null    object 
 2   Job Description    956 non-null    object 
 3   Rating             956 non-null    float64
 4   Company Name       956 non-null    object 
 5   Location           956 non-null    object 
 6   Headquarters       956 non-null    object 
 7   Size               956 non-null    object 
 8   Founded            956 non-null    int64  
 9   Type of ownership  956 non-null    object 
 10  Industry           956 non-null    object 
 11  Sector             956 non-null    object 
 12  Revenue            956 non-null    object 
 13  Competitors        956 non-null    object 
dtypes: float64(1), int64(1), object(12)
memory usage: 112.0+ KB


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1
1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1
2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1
3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa..."
4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


### Data Cleaning
#### In this section we mostly use lambda functions for vanish substrings, split strings.
#### Within some lambda function we use if statement(s) for sorting or conditional splits/replaces/removes.
#### Sometimes we just remove the last x character from a string.
### Feature engineering
#### Extract features like Description length, average salary, required skills (SQL, Excel, etc.), Same State or not, etc.

In [2]:

jobs.columns = jobs.columns.str.replace(' ','_')

jobs['Competitors'] = jobs['Competitors'].apply(lambda x: 0 if x=='-1' else 1)


jobs['Company_Name'] = jobs.apply(lambda x: x['Company_Name'] if x['Rating'] < 0 else x['Company_Name'][:-4],axis=1)

jobs['Size'] = jobs['Size'].str[:-9]

jobs['Revenue'] = jobs['Revenue'].str[:-5]

jobs['Salary_NotHour'] = jobs['Salary_Estimate'].apply(lambda x: 0 if 'per hour' in x.lower() else 1)
jobs['Employer_est.'] = jobs['Salary_Estimate'].apply(lambda x: 1 if 'employer' in x.lower() else 0)

jobs['Size_Bottom'] = jobs['Size'].apply(lambda x: x.split('to')[0] if 'to' in x else(x.replace('+','') if '+' in x else None))
jobs['Size_Top'] = jobs['Size'].apply(lambda x: x.split('to')[1] if 'to' in x else 0)

jobs['Size_Bottom'] = pd.to_numeric(jobs['Size_Bottom'])
jobs['Size_Top'] = pd.to_numeric(jobs['Size_Top'])


jobs['AVG_Size'] = pd.Series(dtype='float64')


for i in range(len(jobs['Size_Bottom'])):
    if jobs['Size_Bottom'][i] < 10000:
        jobs['AVG_Size'][i] = (jobs['Size_Bottom'][i] + jobs['Size_Top'][i])/2
    else: jobs['AVG_Size'][i] = jobs['Size_Bottom'][i]



jobs['Python'] = jobs['Job_Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)

jobs['R'] = jobs['Job_Description'].apply(lambda x: 1 if 'r studio' in x else (1 if 'r-studio' in x.lower() else 0))

jobs['AWS'] = jobs['Job_Description'].apply(lambda x: 1 if 'aws' in x.lower() else 0)

jobs['Excel'] = jobs['Job_Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)

jobs['SQL'] = jobs['Job_Description'].apply(lambda x: 1 if 'sql' in x.lower() else 0)

C:\Users\traue\AppData\Local\Temp\ipykernel_23348\1672353259.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobs['AVG_Size'][i] = (jobs['Size_Bottom'][i] + jobs['Size_Top'][i])/2
C:\Users\traue\AppData\Local\Temp\ipykernel_23348\1672353259.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  else: jobs['AVG_Size'][i] = jobs['Size_Bottom'][i]


In [3]:
jobs['Salary_Bottom'] = jobs['Salary_Estimate'].apply(lambda x: x.replace('$','').replace('K',''))

In [4]:
jobs['Salary_Bottom'] = jobs['Salary_Bottom'].apply(lambda x: x.split('(')[0] if '(' in x else x)

In [5]:
jobs['Salary_Bottom'] = jobs['Salary_Bottom'].apply(lambda x: x.split(':')[1] if ':' in x else x)

In [6]:
jobs['Salary_Bottom'] = jobs['Salary_Bottom'].apply(lambda x: x.replace('Per Hour','') if 'per hour' in x.lower() else x)

In [7]:
jobs['Salary_Range'] = jobs['Salary_Bottom']
jobs['Salary_Bottom'] = jobs['Salary_Bottom'].apply(lambda x: x.split('-')[0])
jobs['Salary_Top'] = jobs['Salary_Range'].apply(lambda x: x.split('-')[1])

In [8]:
jobs['Salary_Bottom'] = pd.to_numeric(jobs['Salary_Bottom'])
jobs['Salary_Top'] = pd.to_numeric(jobs['Salary_Top'])

#### In this for loop we calculate the bottom and top salaries.
##### Reason of the multiplies is that, get the correct numbers.
##### Multiply 1000 if the salary is not hourly. ( 40 -> 40000)
##### Multiply 2080 if the salary is hourly. Here we assume 260 work days in a year and 8 hour long work days. (8*260=2080)

In [9]:
for i in range(len(jobs['Salary_Bottom'])):
    if jobs['Salary_NotHour'][i] == 0:
        jobs['Salary_Bottom'][i] = jobs['Salary_Bottom'][i] *2080
    else: jobs['Salary_Bottom'][i] = jobs['Salary_Bottom'][i] * 1000
#jobs['Salary_Bottom'] = jobs.apply(lambda x: x['Salary_Bottom']*1000 if x['Salary_NotHour'] > 0 else x['Salary_Bottom']*2080)

C:\Users\traue\AppData\Local\Temp\ipykernel_23348\3749585329.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  else: jobs['Salary_Bottom'][i] = jobs['Salary_Bottom'][i] * 1000
C:\Users\traue\AppData\Local\Temp\ipykernel_23348\3749585329.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobs['Salary_Bottom'][i] = jobs['Salary_Bottom'][i] *2080


In [10]:
for i in range(len(jobs['Salary_Top'])):
    if jobs['Salary_NotHour'][i] == 0:
        jobs['Salary_Top'][i] = jobs['Salary_Top'][i] *2080
    else: jobs['Salary_Top'][i] = jobs['Salary_Top'][i] * 1000
#jobs['Salary_Bottom'] = jobs.apply(lambda x: x['Salary_Bottom']*1000 if x['Salary_NotHour'] > 0 else x['Salary_Bottom']*2080)
#jobs['Salary_Top'].value_counts()

C:\Users\traue\AppData\Local\Temp\ipykernel_23348\1642058894.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  else: jobs['Salary_Top'][i] = jobs['Salary_Top'][i] * 1000
C:\Users\traue\AppData\Local\Temp\ipykernel_23348\1642058894.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobs['Salary_Top'][i] = jobs['Salary_Top'][i] *2080


In [11]:
jobs['AVG_Salary'] = (jobs['Salary_Top'] + jobs['Salary_Bottom'])/2

In [12]:
#jobs['Location'][0][-2:]
#jobs['Same_State'] = jobs.apply(lambda x: 1 if x['Location'][-2:]==x['Headquarters'][-2:] else 0)
jobs['Same_State'] = pd.Series(dtype='float64')

for i in range(len(jobs['Location'])):
    if jobs['Location'][i][-2:]==jobs['Headquarters'][i][-2:]:
        jobs['Same_State'][i]=1
    else: jobs['Same_State'][i]=0

C:\Users\traue\AppData\Local\Temp\ipykernel_23348\838272082.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  else: jobs['Same_State'][i]=0
C:\Users\traue\AppData\Local\Temp\ipykernel_23348\838272082.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobs['Same_State'][i]=1


In [13]:
jobs['Descr._Length'] = pd.Series(dtype='float64')
for i in range(len(jobs['Job_Description'])):
    jobs['Descr._Length'][i] = len(jobs['Job_Description'][i])

C:\Users\traue\AppData\Local\Temp\ipykernel_23348\3436497550.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobs['Descr._Length'][i] = len(jobs['Job_Description'][i])


#### Get simpler form of job titles and get seniority.

In [14]:
def get_simpTitle(title):
    if 'data scientist' in title.lower():
        return 'data scientist'
    elif 'analyst' in title.lower():
        return 'analyst'
    elif 'data engineer' in title.lower():
        return 'data engineer'
    elif 'machine learning' in title.lower():
        return 'mle'
    elif 'research' in title.lower():
        return 'research'
    elif 'manager' in title.lower():
        return 'manager'
    elif 'director' in title.lower():
        return 'director'
    else: return 'na'
jobs['Job_simp'] = jobs['Job_Title'].apply(get_simpTitle)

In [15]:
def get_seniority(title):
    if 'junior' in title.lower() or 'jr.' in title.lower() or 'jr' in title.lower():
        return 'junior'
    elif 'medior' in title.lower():
        return 'medior'
    elif 'sr' in title.lower() or 'sr.' in title.lower() or 'senior' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower():
        return 'senior'
    else: return 'na'

jobs['Seniority'] = jobs['Job_Title'].apply(get_seniority)

#### Get dummies, and concat to the original Dataframe
#### Drop rows where Salary Estimmate or Founded is -1
#### Reset index after the drops

In [16]:
#jobs.to_csv('jobs_for_analysis.csv', sep='\t', encoding='utf-8')
#jobs.to_csv('jobs_for_ML.csv', sep='\t', encoding='utf-8')
jobs = pd.concat([jobs, pd.get_dummies(jobs.Job_simp)],axis=1)
jobs = pd.concat([jobs, pd.get_dummies(jobs.Seniority)],axis=1)
jobs = jobs[jobs['Salary_Estimate'] != '-1']
jobs = jobs[jobs['Founded'] != (-1)]
jobs.reset_index(drop=True)

,Job_Title,Salary_Estimate,Job_Description,Rating,Company_Name,Location,Headquarters,Size,Founded,Type_of_ownership,...,data engineer,data scientist,director,manager,mle,na,research,junior,na,senior
0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research,"Albuquerque, NM","Goleta, CA",501 to 1000,1973,Company - Private,...,0,1,0,0,0,0,0,0,1,0
1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System,"Linthicum, MD","Baltimore, MD",10000+,1984,Other Organization,...,0,1,0,0,0,0,0,0,1,0
2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4,"Clearwater, FL","Clearwater, FL",501 to 1000,2010,Company - Private,...,0,1,0,0,0,0,0,0,1,0
3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL,"Richland, WA","Richland, WA",1001 to 5000,1965,Government,...,0,1,0,0,0,0,0,0,1,0
4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,"New York, NY","New York, NY",51 to 200,1998,Company - Private,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687,"Principal, Data Science - Advanced Analytics",$86K-$137K (Glassdoor est.),IQVIA is the leading human data science compan...,3.6,IQVIA,"Plymouth Meeting, PA","Durham, NC",10000+,2017,Company - Public,...,0,0,0,0,0,1,0,0,0,1
688,"Sr Scientist, Immuno-Oncology - Oncology",$58K-$111K (Glassdoor est.),Site Name: USA - Massachusetts - Cambridge\nPo...,3.9,GSK,"Cambridge, MA","Brentford, United Kingdom",10000+,1830,Company - Public,...,0,0,0,0,0,1,0,0,0,1
689,Senior Data Engineer,$72K-$133K (Glassdoor est.),THE CHALLENGE\nEventbrite has a world-class da...,4.4,Eventbrite,"Nashville, TN","San Francisco, CA",1001 to 5000,2006,Company - Public,...,1,0,0,0,0,0,0,0,0,1
690,"Project Scientist - Auton Lab, Robotics Institute",$56K-$91K (Glassdoor est.),The Auton Lab at Carnegie Mellon University is...,2.6,Software Engineering Institute,"Pittsburgh, PA","Pittsburgh, PA",501 to 1000,1984,College / University,...,0,0,0,0,0,1,0,0,1,0


In [17]:
jobs.columns

Index(['Job_Title', 'Salary_Estimate', 'Job_Description', 'Rating',
       'Company_Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type_of_ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'Salary_NotHour', 'Employer_est.', 'Size_Bottom', 'Size_Top',
       'AVG_Size', 'Python', 'R', 'AWS', 'Excel', 'SQL', 'Salary_Bottom',
       'Salary_Range', 'Salary_Top', 'AVG_Salary', 'Same_State',
       'Descr._Length', 'Job_simp', 'Seniority', 'analyst', 'data engineer',
       'data scientist', 'director', 'manager', 'mle', 'na', 'research',
       'junior', 'na', 'senior'],
      dtype='object')

### Create a new Dataframe with relevant (for ML) columns, and save it as csv file. (into the current folder).

In [18]:
jobs_ml = jobs[['Rating', 'Founded', 'Competitors', 'Salary_NotHour', 'Employer_est.', 'AVG_Size', 'Python', 'R', 'AWS', 'Excel',
           'SQL', 'AVG_Salary', 'Same_State', 'Descr._Length', 'analyst', 'data engineer', 'data scientist', 'director',
           'manager', 'mle', 'na', 'research', 'junior', 'na', 'senior','Type_of_ownership', 'Industry', 'Sector', 'Revenue']]

In [19]:
jobs_ml = jobs_ml.reset_index()

In [20]:
jobs_ml.to_csv('jobs_for_ML.csv', sep='\t', encoding='utf-8')